## Importing necessary libraries

In [17]:
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error 
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import warnings 
import tensorflow as tf
import h5py
from sklearn.preprocessing import StandardScaler
import seaborn as sns
from sklearn.metrics import accuracy_score


warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=DeprecationWarning)


np.random.seed(1)
tf.random.set_seed(1)

## Loading the dataset

In [18]:
df = pd.read_csv('asthma_scaled-final.csv',header = 0)



df

,Sampler State,Indoor total spores,Outdoor total spores,Smoking at home,Pre-existing Respiratory Symptoms,Pre-existing Respiratory Severity,Post-Hurricane Respiratory Symptoms,Post-Hurricane Respiratory Severity,Blower Door Test at 50 Pa,Maximum flood depth,How often residences open window blinds,Roof age,Average Bathroom Exhaust Fan Airflow Rates
0,Louisiana,0.857531,0.600928,0,0,0.935478,0,0.000000,0.302308,1.000000,0.375,0.257143,0.293578
1,Louisiana,0.736371,0.523923,1,0,0.000000,1,0.000000,0.167651,0.785714,0.875,0.257143,0.385321
2,Louisiana,0.482403,0.307157,1,1,0.000000,1,0.000000,0.233777,0.880952,0.875,0.371429,0.362385
3,Louisiana,1.000000,0.536898,1,0,0.000000,0,0.732677,0.491702,0.642857,0.625,0.542857,0.351682
4,Louisiana,0.000000,0.806030,1,1,0.000000,1,0.000000,0.455453,0.166667,0.375,0.428571,0.284404
...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,Florida,0.402976,0.591573,0,1,0.000000,1,0.000000,0.072765,0.000000,0.875,0.571429,0.000000
58,Florida,0.589576,0.626633,1,0,0.587859,0,0.000000,0.058077,0.000000,1.000,0.714286,0.000000
59,Florida,0.482403,0.400078,1,1,0.000000,1,0.000000,0.078863,0.000000,1.000,0.571429,0.000000
60,Florida,0.697962,0.884263,0,1,0.000000,1,0.000000,0.051627,0.228571,0.375,0.600000,0.174312


## Selecting important features for the analysis based on correlation matrix and forward feature selection

In [19]:
selected_features =     [
'Indoor total spores', 
'Outdoor total spores',
'Smoking at home', 
# 'Pre-existing Respiratory Symptoms',
# 'Pre-existing Respiratory Severity',
# 'Post-Hurricane Respiratory Severity', 
'Blower Door Test at 50 Pa', 
'Maximum flood depth', 
'How often residences open window blinds', 
'Roof age', 
'Average Bathroom Exhaust Fan Airflow Rates', 
    ]

## Spliting the train and test dataset

In [20]:
from sklearn.model_selection import train_test_split


X = np.float32(df[selected_features].values)
Y= df['Post-Hurricane Respiratory Symptoms']

# Split data into training and testing sets
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, test_size=0.2, random_state=42)

## Classification TSNE  model developement

In [21]:
import numpy as np
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler

# Combine training and test data for t-SNE
scaler = StandardScaler()
X_scaled = scaler.fit_transform(np.vstack((Xtrain, Xtest)))

# Apply t-SNE transformation to the combined data
tsne = TSNE(n_components=2, random_state=1)
X_tsne = tsne.fit_transform(X_scaled)

# Split the t-SNE transformed data back into training and test sets
Xtrain_tsne = X_tsne[:len(Xtrain)]
Xtest_tsne = X_tsne[len(Xtrain):]

# Proceed with defining your model as before
# Define the input layer (based on the transformed 2D data)
inputs = tf.keras.Input(shape=(Xtrain_tsne.shape[1],))

# Define the hidden layers
hidden = tf.keras.layers.Dense(16, activation='relu', kernel_initializer=tf.keras.initializers.he_normal(seed=1),
                               bias_initializer='zeros', kernel_regularizer=tf.keras.regularizers.L1(0.001))(inputs)
hidden = tf.keras.layers.Dense(32, activation='relu', kernel_initializer=tf.keras.initializers.he_normal(seed=1),
                               bias_initializer='zeros', kernel_regularizer=tf.keras.regularizers.L1(0.001))(hidden)

# Define the output layer for binary classification
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(hidden)


# Create the model
model = tf.keras.models.Model(inputs=inputs, outputs=outputs)
model.summary()

# Custom early stopping callback
class CustomEarlyStopping(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        if logs['val_loss'] < 0.94 and logs['loss'] < 0.94:
            self.model.stop_training = True

# Instantiate callbacks
custom_early_stopping = CustomEarlyStopping()
model_checkpoint = ModelCheckpoint('model.h5', monitor='val_loss', save_best_only=True)

# Exponential decay learning rate schedule
lr = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.001,
    decay_steps=150,
    decay_rate=0.92,
    staircase=True
)

# Compile the model for binary classification
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Train the model
history = model.fit(
    Xtrain_tsne,
    Ytrain,
    batch_size=8,
    epochs=1000,
    validation_data=(Xtest_tsne, Ytest),
    callbacks=[custom_early_stopping, model_checkpoint]
)


Model: "model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         [(None, 2)]               0         
_________________________________________________________________
dense_18 (Dense)             (None, 16)                48        
_________________________________________________________________
dense_19 (Dense)             (None, 32)                544       
_________________________________________________________________
dense_20 (Dense)             (None, 1)                 33        
Total params: 625
Trainable params: 625
Non-trainable params: 0
_________________________________________________________________
Epoch 1/1000
7/7 [==============================] - 1s 30ms/step - loss: 4.8057 - accuracy: 0.4898 - val_loss: 5.4100 - val_accuracy: 0.4615
Epoch 2/1000
7/7 [==============================] - 0s 5ms/step - loss: 3.6924 - accuracy: 0.4898 - val_loss: 4.4471 -

Epoch 53/1000
7/7 [==============================] - 0s 6ms/step - loss: 0.7137 - accuracy: 0.7143 - val_loss: 1.0440 - val_accuracy: 0.5385
Epoch 54/1000
7/7 [==============================] - 0s 7ms/step - loss: 0.7075 - accuracy: 0.6531 - val_loss: 1.0758 - val_accuracy: 0.5385
Epoch 55/1000
7/7 [==============================] - 0s 7ms/step - loss: 0.7062 - accuracy: 0.5918 - val_loss: 1.0994 - val_accuracy: 0.5385
Epoch 56/1000
7/7 [==============================] - 0s 6ms/step - loss: 0.6962 - accuracy: 0.6327 - val_loss: 1.0806 - val_accuracy: 0.4615
Epoch 57/1000
7/7 [==============================] - 0s 8ms/step - loss: 0.7079 - accuracy: 0.6531 - val_loss: 1.0529 - val_accuracy: 0.4615
Epoch 58/1000
7/7 [==============================] - 0s 6ms/step - loss: 0.7152 - accuracy: 0.6939 - val_loss: 1.0511 - val_accuracy: 0.5385
Epoch 59/1000
7/7 [==============================] - 0s 9ms/step - loss: 0.7021 - accuracy: 0.6939 - val_loss: 1.0557 - val_accuracy: 0.4615
Epoch 60/1000

7/7 [==============================] - 0s 7ms/step - loss: 0.6779 - accuracy: 0.6531 - val_loss: 1.1919 - val_accuracy: 0.5385
Epoch 112/1000
7/7 [==============================] - 0s 8ms/step - loss: 0.6714 - accuracy: 0.7143 - val_loss: 1.1598 - val_accuracy: 0.6154
Epoch 113/1000
7/7 [==============================] - 0s 8ms/step - loss: 0.6698 - accuracy: 0.6939 - val_loss: 1.1357 - val_accuracy: 0.6923
Epoch 114/1000
7/7 [==============================] - 0s 8ms/step - loss: 0.6612 - accuracy: 0.6735 - val_loss: 1.1292 - val_accuracy: 0.6154
Epoch 115/1000
7/7 [==============================] - 0s 6ms/step - loss: 0.6803 - accuracy: 0.6939 - val_loss: 1.1246 - val_accuracy: 0.5385
Epoch 116/1000
7/7 [==============================] - 0s 6ms/step - loss: 0.6674 - accuracy: 0.6939 - val_loss: 1.1376 - val_accuracy: 0.6923
Epoch 117/1000
7/7 [==============================] - 0s 8ms/step - loss: 0.6659 - accuracy: 0.7143 - val_loss: 1.1624 - val_accuracy: 0.6154
Epoch 118/1000
7/7 [=

Epoch 169/1000
7/7 [==============================] - 0s 6ms/step - loss: 0.6412 - accuracy: 0.6735 - val_loss: 1.1826 - val_accuracy: 0.5385
Epoch 170/1000
7/7 [==============================] - 0s 7ms/step - loss: 0.6525 - accuracy: 0.6531 - val_loss: 1.2324 - val_accuracy: 0.5385
Epoch 171/1000
7/7 [==============================] - 0s 6ms/step - loss: 0.6584 - accuracy: 0.6531 - val_loss: 1.2401 - val_accuracy: 0.4615
Epoch 172/1000
7/7 [==============================] - 0s 6ms/step - loss: 0.6520 - accuracy: 0.6735 - val_loss: 1.2055 - val_accuracy: 0.5385
Epoch 173/1000
7/7 [==============================] - 0s 8ms/step - loss: 0.6418 - accuracy: 0.7551 - val_loss: 1.1977 - val_accuracy: 0.5385
Epoch 174/1000
7/7 [==============================] - 0s 7ms/step - loss: 0.6440 - accuracy: 0.6939 - val_loss: 1.1845 - val_accuracy: 0.5385
Epoch 175/1000
7/7 [==============================] - 0s 6ms/step - loss: 0.6420 - accuracy: 0.6939 - val_loss: 1.1804 - val_accuracy: 0.5385
Epoch 

Epoch 227/1000
7/7 [==============================] - 0s 8ms/step - loss: 0.6260 - accuracy: 0.7551 - val_loss: 1.2077 - val_accuracy: 0.6154
Epoch 228/1000
7/7 [==============================] - 0s 6ms/step - loss: 0.6238 - accuracy: 0.7347 - val_loss: 1.2081 - val_accuracy: 0.6154
Epoch 229/1000
7/7 [==============================] - 0s 8ms/step - loss: 0.6228 - accuracy: 0.7959 - val_loss: 1.2216 - val_accuracy: 0.6154
Epoch 230/1000
7/7 [==============================] - 0s 8ms/step - loss: 0.6272 - accuracy: 0.7755 - val_loss: 1.2299 - val_accuracy: 0.5385
Epoch 231/1000
7/7 [==============================] - 0s 6ms/step - loss: 0.6332 - accuracy: 0.7347 - val_loss: 1.2267 - val_accuracy: 0.6154
Epoch 232/1000
7/7 [==============================] - 0s 6ms/step - loss: 0.6260 - accuracy: 0.7347 - val_loss: 1.2500 - val_accuracy: 0.5385
Epoch 233/1000
7/7 [==============================] - 0s 6ms/step - loss: 0.6318 - accuracy: 0.7143 - val_loss: 1.2427 - val_accuracy: 0.6154
Epoch 

...

Epoch 632/1000
7/7 [==============================] - 0s 8ms/step - loss: 0.5941 - accuracy: 0.7143 - val_loss: 1.2427 - val_accuracy: 0.6154
Epoch 633/1000
7/7 [==============================] - 0s 8ms/step - loss: 0.5958 - accuracy: 0.6939 - val_loss: 1.2402 - val_accuracy: 0.6923
Epoch 634/1000
7/7 [==============================] - 0s 6ms/step - loss: 0.5954 - accuracy: 0.6939 - val_loss: 1.2398 - val_accuracy: 0.6154
Epoch 635/1000
7/7 [==============================] - 0s 8ms/step - loss: 0.5935 - accuracy: 0.7755 - val_loss: 1.2438 - val_accuracy: 0.6154
Epoch 636/1000
7/7 [==============================] - 0s 8ms/step - loss: 0.5931 - accuracy: 0.7755 - val_loss: 1.2550 - val_accuracy: 0.6923
Epoch 637/1000
7/7 [==============================] - 0s 6ms/step - loss: 0.5937 - accuracy: 0.7755 - val_loss: 1.2605 - val_accuracy: 0.6923
Epoch 638/1000
7/7 [==============================] - 0s 6ms/step - loss: 0.5939 - accuracy: 0.7755 - val_loss: 1.2635 - val_accuracy: 0.6923
Epoch 

Epoch 747/1000
7/7 [==============================] - 0s 9ms/step - loss: 0.5910 - accuracy: 0.7755 - val_loss: 1.2703 - val_accuracy: 0.6154
Epoch 748/1000
7/7 [==============================] - 0s 7ms/step - loss: 0.5908 - accuracy: 0.7755 - val_loss: 1.2686 - val_accuracy: 0.6154
Epoch 749/1000
7/7 [==============================] - 0s 7ms/step - loss: 0.5902 - accuracy: 0.7755 - val_loss: 1.2664 - val_accuracy: 0.6154
Epoch 750/1000
7/7 [==============================] - 0s 8ms/step - loss: 0.5898 - accuracy: 0.7551 - val_loss: 1.2630 - val_accuracy: 0.6154
Epoch 751/1000
7/7 [==============================] - 0s 7ms/step - loss: 0.5895 - accuracy: 0.7551 - val_loss: 1.2632 - val_accuracy: 0.6154
Epoch 752/1000
7/7 [==============================] - 0s 6ms/step - loss: 0.5896 - accuracy: 0.7551 - val_loss: 1.2655 - val_accuracy: 0.6154
Epoch 753/1000
7/7 [==============================] - 0s 6ms/step - loss: 0.5895 - accuracy: 0.7755 - val_loss: 1.2680 - val_accuracy: 0.6154
Epoch 

Epoch 805/1000
7/7 [==============================] - 0s 6ms/step - loss: 0.5880 - accuracy: 0.7551 - val_loss: 1.2577 - val_accuracy: 0.6154
Epoch 806/1000
7/7 [==============================] - 0s 9ms/step - loss: 0.5891 - accuracy: 0.7551 - val_loss: 1.2581 - val_accuracy: 0.6154
Epoch 807/1000
7/7 [==============================] - 0s 6ms/step - loss: 0.5888 - accuracy: 0.7551 - val_loss: 1.2566 - val_accuracy: 0.6154
Epoch 808/1000
7/7 [==============================] - 0s 6ms/step - loss: 0.5890 - accuracy: 0.7551 - val_loss: 1.2567 - val_accuracy: 0.6154
Epoch 809/1000
7/7 [==============================] - 0s 9ms/step - loss: 0.5888 - accuracy: 0.7551 - val_loss: 1.2585 - val_accuracy: 0.6154
Epoch 810/1000
7/7 [==============================] - 0s 8ms/step - loss: 0.5889 - accuracy: 0.7551 - val_loss: 1.2595 - val_accuracy: 0.6154
Epoch 811/1000
7/7 [==============================] - 0s 6ms/step - loss: 0.5887 - accuracy: 0.7755 - val_loss: 1.2603 - val_accuracy: 0.6154
Epoch 

7/7 [==============================] - 0s 6ms/step - loss: 0.5866 - accuracy: 0.7755 - val_loss: 1.2619 - val_accuracy: 0.6154
Epoch 920/1000
7/7 [==============================] - 0s 8ms/step - loss: 0.5868 - accuracy: 0.7755 - val_loss: 1.2628 - val_accuracy: 0.6154
Epoch 921/1000
7/7 [==============================] - 0s 6ms/step - loss: 0.5866 - accuracy: 0.7755 - val_loss: 1.2633 - val_accuracy: 0.6154
Epoch 922/1000
7/7 [==============================] - 0s 8ms/step - loss: 0.5867 - accuracy: 0.7755 - val_loss: 1.2651 - val_accuracy: 0.6154
Epoch 923/1000
7/7 [==============================] - 0s 8ms/step - loss: 0.5865 - accuracy: 0.7755 - val_loss: 1.2646 - val_accuracy: 0.6154
Epoch 924/1000
7/7 [==============================] - 0s 7ms/step - loss: 0.5868 - accuracy: 0.7755 - val_loss: 1.2643 - val_accuracy: 0.6154
Epoch 925/1000
7/7 [==============================] - 0s 7ms/step - loss: 0.5863 - accuracy: 0.7755 - val_loss: 1.2657 - val_accuracy: 0.6154
Epoch 926/1000
7/7 [=

Epoch 977/1000
7/7 [==============================] - 0s 8ms/step - loss: 0.5858 - accuracy: 0.7551 - val_loss: 1.2623 - val_accuracy: 0.6154
Epoch 978/1000
7/7 [==============================] - 0s 6ms/step - loss: 0.5858 - accuracy: 0.7755 - val_loss: 1.2616 - val_accuracy: 0.6154
Epoch 979/1000
7/7 [==============================] - 0s 6ms/step - loss: 0.5858 - accuracy: 0.7551 - val_loss: 1.2609 - val_accuracy: 0.6154
Epoch 980/1000
7/7 [==============================] - 0s 7ms/step - loss: 0.5862 - accuracy: 0.7551 - val_loss: 1.2600 - val_accuracy: 0.6154
Epoch 981/1000
7/7 [==============================] - 0s 9ms/step - loss: 0.5861 - accuracy: 0.7551 - val_loss: 1.2600 - val_accuracy: 0.6154
Epoch 982/1000
7/7 [==============================] - 0s 8ms/step - loss: 0.5860 - accuracy: 0.7551 - val_loss: 1.2605 - val_accuracy: 0.6154
Epoch 983/1000
7/7 [==============================] - 0s 8ms/step - loss: 0.5861 - accuracy: 0.7551 - val_loss: 1.2612 - val_accuracy: 0.6154
Epoch 

## Model evaluation for the train and test

In [23]:
from sklearn.metrics import accuracy_score
import numpy as np

# Predict and evaluate with t-SNE transformed features
y_pred_train = model.predict(Xtrain_tsne)
y_pred_test = model.predict(Xtest_tsne)

# Convert predicted probabilities to binary labels
y_pred_train_binary = np.round(y_pred_train).astype(int)
y_pred_test_binary = np.round(y_pred_test).astype(int)

# Calculate accuracy using the converted binary labels
train_accuracy = accuracy_score(Ytrain, y_pred_train_binary)
test_accuracy = accuracy_score(Ytest, y_pred_test_binary)

print(f"Training Accuracy: {train_accuracy:.3f}")
print(f"Test Accuracy: {test_accuracy:.3f}")


Training Accuracy: 0.755
Test Accuracy: 0.615


## Saving the model

In [24]:
# Specify the path and filename for the model
model_path = 'asthma_TSNE_final.h5'

# Use the .save() method to save your model
model.save(model_path)

print(f"Model saved to {model_path}")


Model saved to asthma_TSNE.h5


## Calculating confusion matrix and specificity and sendsitivity

In [25]:
from sklearn.metrics import confusion_matrix

# Calculate confusion matrix for training set
cm_train = confusion_matrix(Ytrain, y_pred_train_binary)

# Calculate confusion matrix for test set
cm_test = confusion_matrix(Ytest, y_pred_test_binary)

# Calculate specificity and sensitivity for training set
tn_train, fp_train, fn_train, tp_train = cm_train.ravel()
specificity_train = tn_train / (tn_train + fp_train)
sensitivity_train = tp_train / (tp_train + fn_train)

# Calculate specificity and sensitivity for test set
tn_test, fp_test, fn_test, tp_test = cm_test.ravel()
specificity_test = tn_test / (tn_test + fp_test)
sensitivity_test = tp_test / (tp_test + fn_test)

print("Confusion Matrix - Training Set:")
print(cm_train)
print("\nSpecificity - Training Set:", specificity_train)
print("Sensitivity - Training Set:", sensitivity_train)

print("\nConfusion Matrix - Test Set:")
print(cm_test)
print("\nSpecificity - Test Set:", specificity_test)
print("Sensitivity - Test Set:", sensitivity_test)


Confusion Matrix - Training Set:
[[20  5]
 [ 7 17]]

Specificity - Training Set: 0.8
Sensitivity - Training Set: 0.7083333333333334

Confusion Matrix - Test Set:
[[5 2]
 [3 3]]

Specificity - Test Set: 0.7142857142857143
Sensitivity - Test Set: 0.5


## Calculating F1 score and F2 score

In [26]:
from sklearn.metrics import f1_score

# Calculate F1 score for both sets
f1_train = f1_score(Ytrain, y_pred_train_binary)
f1_test = f1_score(Ytest, y_pred_test_binary)

print("\nF1 Score - Training Set:", f1_train)
print("F1 Score - Test Set:", f1_test)

# Calculate F2 score for both sets
beta = 2  # Weighting factor for recall
f2_train = (1 + beta**2) * (specificity_train * sensitivity_train) / ((beta**2 * specificity_train) + sensitivity_train)
f2_test = (1 + beta**2) * (specificity_test * sensitivity_test) / ((beta**2 * specificity_test) + sensitivity_test)

print("\nF2 Score - Training Set:", f2_train)
print("F2 Score - Test Set:", f2_test)



F1 Score - Training Set: 0.7391304347826088
F1 Score - Test Set: 0.5454545454545454

F2 Score - Training Set: 0.7249466950959489
F2 Score - Test Set: 0.5319148936170213


## Calculating AUC and False alarm rate

In [27]:
from sklearn.metrics import roc_auc_score, confusion_matrix
import numpy as np

# Predict probabilities on t-SNE transformed data
y_pred_train_probabilities = model.predict(Xtrain_tsne)
y_pred_test_probabilities = model.predict(Xtest_tsne)

# Calculate AUC for both sets
auc_train = roc_auc_score(Ytrain, y_pred_train_probabilities)
auc_test = roc_auc_score(Ytest, y_pred_test_probabilities)

print("\nAUC - Training Set:", auc_train)
print("AUC - Test Set:", auc_test)

# Calculate confusion matrices to get True Negatives (TN) and False Positives (FP)
tn_train, fp_train, fn_train, tp_train = confusion_matrix(Ytrain, np.round(y_pred_train_probabilities)).ravel()
false_alarm_rate_train = fp_train / (fp_train + tn_train)

tn_test, fp_test, fn_test, tp_test = confusion_matrix(Ytest, np.round(y_pred_test_probabilities)).ravel()
false_alarm_rate_test = fp_test / (fp_test + tn_test)

print("\nFalse Alarm Rate - Training Set:", false_alarm_rate_train)
print("False Alarm Rate - Test Set:", false_alarm_rate_test)



AUC - Training Set: 0.8683333333333333
AUC - Test Set: 0.5952380952380952

False Alarm Rate - Training Set: 0.2
False Alarm Rate - Test Set: 0.2857142857142857
